In [1]:
import torch
import os
import pandas as pd
import pickle
import numpy as np

In [2]:
ls data/parsed

ciao-712      epinion-l-1-100  ml-1m-l-1-100   pinterest-712-     sk-new
ciao-712-     epinion-new      ml-1m-new       pinterest-l-1-100
ciao-new      melon-712        ml-20m-712      pinterest-new
epinion-712   melon-l-1-100    ml-20m-l-1-100  sk-712
epinion-712-  ml-1m-712        pinterest-712   sk-712-


In [186]:
with open("data/parsed/ml-20m-712", 'rb') as f:
    tr, val, te = pickle.load(f)

In [187]:
n_users, n_items = tr.shape

In [188]:
t = (tr+val+te).tocsr()

In [189]:
n_users, n_items, t.nnz

(136675, 15525, 9983776)

In [190]:
t.nnz / (n_users*n_items)

0.004705157191133921

In [101]:
t.data[:] = 1

In [102]:
avg_consumed_items_per_user = t.sum(1).A1.ravel().mean()

In [196]:
avg_consumed_users_per_item = t.sum(0).A1.ravel().mean()

2454.16378383357

In [104]:
avg_consumed_items_per_user, avg_consumed_users_per_item

(39.23903674327488, 50.55675396151147)

In [198]:
std_items_per_user = t.sum(1).A1.ravel().std()
std_users_per_item = t.sum(0).A1.ravel().std()

In [199]:
std_items_per_user, std_users_per_item

(102.3205169341753, 2454.16378383357)

In [78]:
item_clickcnts = -np.sort(-t.sum(0).A1.ravel())

In [79]:
# tot_cnt

In [80]:
tot_cnt = item_clickcnts.sum()
i = 0
tc = 0 
for i in range(n_items // 20):
    tc += item_clickcnts[i]
print("%0.2f" % (100 *(tc / tot_cnt)))

38.80


In [15]:
ls best_res/sk-712

 cdae    'k=1-dot-1'  'k=3-3'      'k=5-dot-5'   torch-warp
 CML     'k=2-2'      'k=3-dot-3'   ours-256-3   vae
'k=1-1'  'k=2-dot-2'  'k=5-5'       ours-2-dot   warp


In [16]:
x = torch.load("saved_models/sk-712/k=3-3", map_location='cuda:0')

FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/sk-712/k=3-3'

In [77]:
x

{'model': mfrec(
   (u_emb): Embedding(15563, 128)
   (i_emb): Embedding(28806, 128)
 ),
 'eval_metric': 'recall',
 'epoch': 150,
 'lr': 0.05,
 'dim': 128,
 'alpha': 0.05,
 'tau': 1.0,
 'reg': 5.0,
 'best': 0.2799716684038024}

In [351]:
x = torch.load("best_res/sk-712/k=1-1")

In [355]:
x.keys()

dict_keys(['model', 'param'])

In [356]:
x['param']['test_at_5'][0]

{'precision': 0.08645664654792116,
 'recall': 0.11059288045846234,
 'map': 0.05806737327641283,
 'ndcg': 0.12764118745257053}

In [89]:
x['test_at_10'][0]

{'precision': 0.23989772115550442,
 'recall': 0.23490268288062535,
 'map': 0.12866754551422305,
 'ndcg': 0.2761454085913488}

In [90]:
x['test_at_20'][0]

{'precision': 0.18205110951605583,
 'recall': 0.3347923257009072,
 'map': 0.07365179328976347,
 'ndcg': 0.2573312567829422}

In [91]:
x['test_at_30'][0]

{'precision': 0.15089593675859236,
 'recall': 0.4008718996359259,
 'map': 0.05070783533983877,
 'ndcg': 0.25274035441075704}

In [92]:
x['test_at_50'][0]

{'precision': 0.1158946408278979,
 'recall': 0.48743766062942645,
 'map': 0.03021926351309031,
 'ndcg': 0.2510542442585091}

In [367]:
for kk in [5, 10, 20, 30, 50]:
    for k,v in x['param']['test_at_%d' % kk][4].items():
        print(v)

0.08636832032975532
0.11055136998741383
0.0581363167967035
0.1278394777486839
0.06624466362432338
0.15763690615216677
0.03612592269352951
0.1395093863729476
0.048752392168408515
0.2131961870338199
0.021531453576039607
0.15493182185973559
0.04033809313509338
0.250757842176916
0.015760604530442063
0.16526361341552692
0.03106874723980548
0.3047671269966551
0.0104692617347614
0.17931094512483467
